## Get judges from case

Get cases json from the API, return all judges from that case

Map the judges to a CourtListener URL

In [1]:
import re
import os
import sys
sys.path.append('..')

import requests

from config import settings
import utils

In [2]:
url = utils.get_api_url() + 'cases/?jurisdiction=ill&full_case=true&format=json&search=rainbow'
headers = {'AUTHORIZATION': 'Token {}'.format(settings.API_KEY)}
response = requests.get(url, headers=headers)
res = response.json()

In [3]:
cases = res['results']
print(len(cases), "results found")

100 results found


In [4]:
not_useful = ["memorandum", "chief", "circuit","justice", "per curiam","senior", "district", "judge",  "justice","and", "associate","mr", "mrs", "ms", "presiding","opinion","the court", "dissenting"]
courtlistener_people_api = "https://www.courtlistener.com/api/rest/v3/people/"
def get_author_name(idx, opinion):
    judges = []
    try:
        authors = opinion['author'].split(', ')
        for author in authors:
            original_author = author
            author_name = fix_name(author)
            first_name = ''
            name_parts = author_name.split(' ')
            if len(name_parts) > 1:
                author_name = name_parts[-1]
                if len(name_parts[0]) > 1:
                    first_name = name_parts[0]
            if len(author_name) > 1:
                author_name = author_name.capitalize() 
                if '-' in author_name:
                    # keep hyphenations in people's names, capitalize each hyphenated part
                    name_parts = author_name.split('-')
                    author_name = ''
                    for idx,part in enumerate(name_parts):
                        if idx < len(name_parts) - 1:
                            author_name += part.capitalize() + '-' 
                        else:
                            author_name += part.capitalize()
                url = "%s?name_last=%s" % (courtlistener_people_api, author_name)
                if first_name:
                    url += "&name_first=%s" % first_name.capitalize()
                judges.append([idx, opinion['type'], url, original_author, author_name, first_name.capitalize()])
    except:
        # for one reason or another the opinion doesn't have an author. 
        # Sometimes it's because the judges have decided that case is not to be treated as precedential
        pass
    return judges

def fix_name(name):
    name = re.sub('[^0-9a-zA-Z\-\s]+', '', name.lower())
    for nu in not_useful:
        name = name.replace(nu,'')    
    return name.strip()


def get_all_judges(cases, opinion_type=None):
    judge_results = []
    for case in cases:

        opinions = case['casebody']['data']['opinions']

        for opinion in opinions:
            if opinion_type:
                if opinion['type'] == opinion_type:
                    names = get_author_name(case['id'],opinion)
            else:
                names = get_author_name(case['id'],opinion)
            if names:
                judge_results += names
    return judge_results
judges = get_all_judges(cases)

# print first five
print(judges[0:5])

[[25411, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Theis', 'PRESIDING JUSTICE THEIS', 'Theis', ''], [173531, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Wolfson', 'JUSTICE WOLFSON', 'Wolfson', ''], [182741, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Divito', 'JUSTICE DiVITO', 'Divito', ''], [243891, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Freeman', 'CHIEF JUSTICE FREEMAN', 'Freeman', ''], [243891, 'concurrence', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Heiple', 'JUSTICE HEIPLE,', 'Heiple', '']]


## DataFrames!

In [5]:
import pandas as pd
judge_df = pd.DataFrame(data=judges, columns=["id","type","best_guess_url","original", "last_name","first_name"])
judge_df.head()

,id,type,best_guess_url,original,last_name,first_name
0,25411,majority,https://www.courtlistener.com/api/rest/v3/peop...,PRESIDING JUSTICE THEIS,Theis,
1,173531,majority,https://www.courtlistener.com/api/rest/v3/peop...,JUSTICE WOLFSON,Wolfson,
2,182741,majority,https://www.courtlistener.com/api/rest/v3/peop...,JUSTICE DiVITO,Divito,
3,243891,majority,https://www.courtlistener.com/api/rest/v3/peop...,CHIEF JUSTICE FREEMAN,Freeman,
4,243891,concurrence,https://www.courtlistener.com/api/rest/v3/peop...,"JUSTICE HEIPLE,",Heiple,
